In [4]:
import cv2
import numpy as np

In [5]:
def grayscale(img): # 흑백이미지로 변환
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def gaussian_blur(img, kernel_size): # 가우시안 필터 ==> 이미지에서 노이즈 제거
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold): # Canny 알고리즘
    return cv2.Canny(img, low_threshold, high_threshold)

def ROI(img,vertices,ignore_color): #관심영역 설정
    mask = np.zeros_like(img) #같은 크기의 검은색으로 채운 이미지
    cv2.fillPoly(mask, vertices, ignore_color)#다각형 그리기(vertices = 다각형 좌표)
    masked_img = cv2.bitwise_and(img,mask)
    return masked_img

def draw_lines(img, lines): #선 그리기
    color = [0,255,0]
    thickness = 3
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1,y1), (x2,y2), color, thickness)

#허프라인 그리기
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength = min_line_len,
                            maxLineGap = max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3) , dtype = np.uint8)
    draw_lines(line_img, lines)
    return line_img

def masking(img, initial_img):
    a = 0.8
    b = 1.
    c = 0.
    return cv2.addWeighted(initial_img,a,img,b,c)

In [6]:
#동영상 파일 받아오기
video_name = "테스트영상3.mp4"
capture = cv2.VideoCapture(video_name)
img_shape = capture.shape #영상 shape

#허프라인 검출 설정
rho = 2 #원점에서 직선까지의 거리를 얼마씩 증가시키면서 조사할것인지
theta = np.pi/180
threshold = 100 #직선이라고 판단하기 위한 최소 교차점(hough space에서)
min_line_len = 50 #하나의 선이라 판단하는 최소 길이
max_line_gap = 100 #선위의 점들 사이의 최대 길이(이거 넘어가면 다른 선이라 판단)

#vertices (관심영역 설정), 채우는 색
vertices = np.array([[(100,img_shape[0]),
                          (540,360),
                          (690,360),
                          (img_shape[1]-100,img_shape[0])]], dtype=np.int32)
fill_color = 255

#영상 재생 및 허프라인 변환
while True:
    #동영상의 전체 프레임수와 동영상의 현재 프레임수가 같으면 영상을 연다.
    if(capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
        capture.open(video_name)
        
    ret, frame = capture.read() #열기
    
    height, width = frame.shape[:2] # 이미지 높이, 너비
    gray_img = grayscale(frame)     # 흑백이미지로 변환
    blur_img = gaussian_blur(gray_img, 13) # Blur 효과
    canny_img = canny(blur_img,50,150) #선 따기
    roi_img = ROI(canny_img,vertices,fill_color) #관심영역 설정
    line_img = hough_lines(roi_img, rho, theta, threshold, min_line_len, max_line_gap) #검은 배경에 라인 따기
    result_img = masking(line_img,frame) #실제 영상에 라인 넣기
    
    #cv2.imshow("BlurFrame",blur_img)
    #cv2.imshow("CannyFrame",canny_img)
    #cv2.imshow("maskFrame",roi_img)
    #cv2.imshow("lineFrame",line_img)
    cv2.imshow("resultFrame",result_img)

    
    if cv2.waitKey(33) & 0xFF == ord('q'):#q누르면 창 끄기, 1ms마다 프레임 재생
        break

capture.release()
#영상 중지 됨 ==> 메모리 프리
cv2.destroyAllWindows()
#이미지 창 모두 끄기

TypeError: 'NoneType' object is not iterable